# Inference from any pose

With this notebook you can run inference of HOOD starting with a garment in any pose. For the body sequence you can use either:
* Sequence of SMPL parameters (see `$HOOD_DATA/fromanypose/pose_sequence.pkl` as an example) or
* Sequence of meshes (see `$HOOD_DATA/fromanypose/mesh_sequence.pkl` as an example)

You also need to have a garment mesh (as `.obj` file) which is aligned with the first frame of your body sequence.

In this notebook we first show how to convert a garment mesh in `.obj` format into a `.pkl` template used in HOOD. 

Then, we show how to use the configuration file `aux/from_any_pose.yaml` to run inference with over albitrary SMPL sequence and arbitrary mesh sequence.

Note that, pinned vertices are not yet supported in this example, so do not expect it to work with lower body garments (they would slide down :c)

## Set environmental variables

In [ ]:
import os

HOOD_PROJECT = "/path/to/hood/repository"
HOOD_DATA = "/path/to/hood/data"

os.environ["HOOD_PROJECT"] = HOOD_PROJECT
os.environ["HOOD_DATA"] = HOOD_DATA

## Create template file from the .obj file:

Use `utils.mesh_creation::obj2template()` function to convert an `.obj` file into a template dictionary and then save it with `pickle_dump`

`$HOOD_DATA/fromanypose/tshirt.obj` is provided as an example

In [ ]:
import trimesh
from utils.mesh_creation import obj2template

In [ ]:
obj_path = Path(DEFAULTS.data_root) / 'fromanypose' / 'tshirt.obj'
out_template_path = Path(DEFAULTS.data_root) / 'fromanypose' / 'tshirt.pkl'

template_dict = obj2template(obj_path)
pickle_dump(template_dict, out_template_path)

## Inference with a SMPL sequence

Here we use a sequence of SMPL parameters as a body sequence. The garment template needs to be aligned with the first frame of the sequence.

The SMPL pose sequence has to be a `.pkl` file containing a dictionary of the following items (see `$HOOD_DATA/fromanypose/pose_sequence.pkl` as an example):
* `body_pose`: np.array [N, 69]
* `global_orient`: np.array [N, 3]
* `transl`: np.array [N, 3]
* `betas`: np.array [10,]

### Edit `aux/from_any_pose` config

First, change the configuration file `configs/aux/from_any_pose.yaml` to set your garment template and the pose sequence

Go to `configs/aux/from_any_pose.yaml` and edit following fields there:

- `dataloader.from_any_pose.scanreg.pose_sequence_type`, type of the pose sequence can be either `smpl` or `mesh`. For this example, set `smpl`
- `dataloader.from_any_pose.scanreg.pose_sequence_path` path to the smpl pose sequence relative to `$HOOD_DATA`. For this example we use `fromanypose/pose_sequence.pkl`
- `dataloader.from_any_pose.scanreg.garment_template_path` path to the .pkl file with the garment template  For this example we use `fromanypose/tshirt.pkl`

### Load runner and dataloader from `from_any_pose` config

In [ ]:
from utils.validation import Config as ValidationConfig
from utils.arguments import load_params, create_modules
from utils.arguments import load_params
from utils.common import move2device, pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path
import torch

In [ ]:
modules, config = load_params('aux/from_any_pose')
dataloader_m, runner_module, runner, aux_modules = create_modules(modules, config)
dataloader = dataloader_m.create_dataloader()

checkpoint_path = Path(DEFAULTS.data_root) / 'trained_models' / 'postcvpr.pth'
state_dict =  torch.load(checkpoint_path)
runner.load_state_dict(state_dict['training_module'])

### load sample, infer and save trajectories

To visualise saved trajectories, see `Inference.ipynb::write_video`

In [ ]:
sample = next(iter(dataloader))

trajectories_dict = runner.valid_rollout(sample)
# Save the sequence to disc
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output_f.pkl'
print(f"Rollout saved into {out_path}")
pickle_dump(dict(trajectories_dict), out_path)

## Inference with a mesh sequence

Here we use a sequence of arbitrary meshes.  The garment template needs to be aligned with the first frame of the sequence.

The mesh sequence has to be a `.pkl` file containing a dictionary of the following items (see `$HOOD_DATA/fromanypose/mesh_sequence.pkl` as an example):
* `verts`: np.array [N, 3]
* `faces`: np.array [F, 3]

### Edit `aux/from_any_pose` config

Change the configuration file `configs/aux/from_any_pose.yaml` to set your garment template and the pose sequence

Go to `configs/aux/from_any_pose.yaml` and edit following fields there:

- `dataloader.from_any_pose.scanreg.pose_sequence_type`, type of the pose sequence can be either `smpl` or `mesh`. For this example, set `mesh`
- `dataloader.from_any_pose.scanreg.pose_sequence_path` path to the smpl pose sequence relative to `$HOOD_DATA`. For this example we use `fromanypose/mesh_sequence.pkl`
- `dataloader.from_any_pose.scanreg.garment_template_path` path to the .pkl file with the garment template  For this example we use `fromanypose/tshirt.pkl`

### Load runner and dataloader from `from_any_pose` config

In [ ]:
from utils.validation import Config as ValidationConfig
from utils.arguments import load_params, create_modules
from utils.arguments import load_params
from utils.common import move2device, pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path
import torch

In [ ]:
modules, config = load_params('aux/from_any_pose')
dataloader_m, runner_module, runner, aux_modules = create_modules(modules, config)
dataloader = dataloader_m.create_dataloader()

checkpoint_path = '/mnt/sdb1/hood_public/trained_models/postcvpr.pth'
state_dict =  torch.load(checkpoint_path)
runner.load_state_dict(state_dict['training_module'])

### load sample, infer and save trajectories

To visualise saved trajectories, see `Inference.ipynb::write_video`

In [ ]:
sample = next(iter(dataloader))

trajectories_dict = runner.valid_rollout(sample)
# Save the sequence to disc
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output_m.pkl'
print(f"Rollout saved into {out_path}")
pickle_dump(dict(trajectories_dict), out_path)